In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from functionst import * 
import numpy as np
%matplotlib inline

In [29]:
# Dataset1: Client Profiles: Demographics like age, gender, and account details of our clients.

In [30]:
file_path='data/df_final_demo.txt'
df_clients = pd.read_csv(file_path, delimiter=',')

In [31]:
# Dataset2: Digital Footprints pt1 & pt2: 
# A detailed trace of client interactions online, divided into two parts: pt_1 and pt_2. 
# It’s recommended to merge these two files prior to a comprehensive data analysis.
file_path='data/df_final_web_data_pt_1.txt'
df2 = pd.read_csv(file_path, delimiter=',')
file_path = "Data/df_final_web_data_pt_2.txt"
df3 = pd.read_csv(file_path, delimiter=',')
df_concat = pd.concat([df2, df3])  # Concatenated 

In [32]:
#Dataset 3: Experiment Roster: A list revealing which clients were part of the grand experiment.
file_path='data/df_final_experiment_clients.txt'
df_roster = pd.read_csv(file_path, delimiter=',')

Cleaning df_Clients

Check for Duplicates

In [ ]:
df_clients.duplicated().sum() # Check

In [34]:
# Rename columns
df_clients.rename(columns={"clnt_tenure_yr": "client_tenure_yr",
                           "clnt_tenure_mnth":"client_tenure_mnth",
                           "clnt_age":"client_age",
                           "gendr":"gender",
                           "bal":"balance"
                           }, inplace=True)

In [ ]:
# Checking for NaN Rows
nan_rows = df_clients[df_clients.isna().any(axis=1)]
# Drop NaN Values with more many NaNs
df_clients.dropna(thresh=4,inplace=True)
# Fill NaN in client age
df_clients["client_age"].fillna(df_clients["client_age"].mean(),inplace=True)

# Drop NaN Values with more many NaNs 
df_clients.dropna(thresh=4,inplace=True)

# Fill NaN in client age
df_clients["client_age"].fillna(df_clients["client_age"].mean(),inplace=True)

# Filled the Client tenure yr with NaN when Client age < then tenure
df_clients.loc[df_clients["client_tenure_yr"] > df_clients["client_age"], "client_tenure_yr"] = np.nan

In [36]:
# Defining age Group and Assigning to client Data

df_clients["client_age_group"] = df_clients["client_age"].apply(age_group)

Clean df_concat

In [37]:
# Transformed DATETIME
df_concat["date_time"] = pd.to_datetime(df_concat["date_time"])

In [ ]:
df_concat.duplicated().sum()

Joined Clients and Joined DC_Concat with Roster

In [39]:
# Joined Clients with the Roster an dropped the NaN Values
df_client_joined = pd.merge(df_clients,df_roster, how="left", on="client_id")

In [40]:
# Joined DF Concat with the Roster and dropped the duplicates
df_concat_joined = pd.merge(df_concat,df_roster, on="client_id", how="left")
df_concat_joined.dropna(subset="Variation",inplace=True)
df_concat_joined.drop_duplicates(inplace=True)

In [ ]:
# Add a column if the client finished the process

# Identify the client_ids that have a "finished" step
clients_with_finished = df_concat_joined[df_concat_joined['process_step'] == 'confirm']['client_id']

# Using map to replace boolean values
df_client_joined['finished'] = df_client_joined["client_id"].isin(clients_with_finished).map({True:"yes",False:"no"})

df_client_joined["finished"].unique()

Who are the clients

In [ ]:
# Who are the clients? By Age
most_clients = df_clients.groupby("client_age_group")["logons_6_mnth"].sum().sort_values( ascending=False)
most_clients

In [ ]:
# Checking 
sns.barplot(data= most_clients)
plt.title("age groups by age")
plt.xlabel("age groups")
plt.ylabel("logins last 6 month")

In [17]:
# Who are the clients by tenure

In [ ]:
sns.boxplot(data=df_clients["client_tenure_yr"])

In [ ]:
sns.violinplot(data=df_clients["client_tenure_yr"])

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_clients['client_age'], df_clients['client_tenure_yr'], color='blue', alpha=0.7)
plt.xlabel('Client Age', fontsize=12)
plt.ylabel('Client Tenure (Years)', fontsize=12)
plt.title('Client Age vs. Client Tenure (Years)', fontsize=15)

Sorting df_concat and trying to calculate time of steps

In [ ]:
# Joined Clients with the Roster an dropped the NaN Values
df_client_joined = pd.merge(df_clients,df_roster, how="left", on="client_id")
df_client_joined_cleaned = df_client_joined.dropna()
df_client_joined_cleaned.head()

In [45]:
# Joined DF Concat with the Roster and dropped the duplicates
df_concat_joined = pd.merge(df_concat,df_roster, on="client_id", how="left")
df_concat_joined.dropna(subset="Variation",inplace=True)
df_concat_joined.drop_duplicates(inplace=True)

In [ ]:
df_concat_joined

In [47]:
df_concat_joined['year_month'] = df_concat_joined['date_time'].dt.to_period('M')

Who are the clients

In [ ]:
# Who are the clients? By Age
most_clients = df_clients.groupby("client_age_group")["logons_6_mnth"].sum().sort_values( ascending=False)
most_clients

In [ ]:
# Checking 
sns.barplot(data= most_clients)
plt.title("age groups by age")
plt.xlabel("age groups")
plt.ylabel("logins last 6 month")
plt.xticks(rotation= 90)

In [51]:
# Who are the clients by tenure

In [ ]:
sns.boxplot(data=df_clients["client_tenure_yr"])

In [ ]:
sns.violinplot(data=df_clients["client_tenure_yr"])

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_clients['client_age_group'], df_clients['client_tenure_yr'], color='blue', alpha=0.7)
plt.xlabel('Client Age', fontsize=12)
plt.ylabel('Client Tenure (Years)', fontsize=12)
plt.title('Client Age vs. Client Tenure (Years)', fontsize=15)

In [ ]:
# Df_concat Sorted and time_diff_in_minutes

# With this we are able to analyze
df_concat_joined_sorted = df_concat_joined.sort_values(by=['client_id', 'visit_id', 'date_time'])
df_concat_joined_sorted['time_diff_in_minutes'] = df_concat_joined_sorted.groupby(['client_id', 'visit_id'])['date_time'].diff().dt.total_seconds() / 60



In [ ]:
# investigating number of calls by age 
sns.boxplot(x='Variation', y='calls_6_mnth', data=df_client_joined_cleaned)

In [57]:
average_calls_by_age_variation = df_client_joined_cleaned.groupby(['client_age_group', 'Variation'])['calls_6_mnth'].mean().reset_index()

In [ ]:
# How many reached
df_concat_joined_sorted["process_step"].value_counts()

In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(x='client_age_group', y='calls_6_mnth', hue='Variation', data=average_calls_by_age_variation)
plt.title('average calls by age and variation')
plt.xlabel('age')
plt.ylabel('average calls last six months')
plt.legend(title='Variation')
plt.grid(True)
plt.xticks(rotation= 90)
plt.show()

In [60]:
# HYPOTHESESTEST: Two Sample T-test
# HO no difference in number of calls between Test & Control
# H1 there is difference between number of calls between Test & Control 
import scipy.stats as st

In [387]:
# Adding a column for every visit which led to finished and go through all 4 steps
finished_list = filter_complete_visit(df_concat_joined_sorted)
df_concat_joined_sorted["finished"] = df_concat_joined_sorted["visit_id"].apply(lambda x:"yes" if x in finished_list else "no" )
len(finished_list)  
df_concat_joined_sorted[df_concat_joined_sorted["finished"] == "yes"]["visit_id"].nunique()

In [ ]:
st.ttest_ind(df_Test_calls,df_Control_calls, equal_var=False)

In [391]:
# This is only to create a csv File
df_concat_joined_sorted.to_csv("Test.csv")

In [ ]:
SD_pooled = np.sqrt(((n1 - 1) * SD1**2 + (n2 - 1) * SD2**2) / (n1 + n2 - 2))
cohens_d = (M1 - M2) / SD_pooled
cohens_d

In [395]:
# p-value < 0.05: assuming there is no difference it is not likely to see no difference in sample: reject null hypotheses.
# effectsize: -0.02 very small, not relevant

## 2.KPI: User Satisfaction Based on Logins in the Last 6 Months
Description: Evaluate user satisfaction through the frequency of user logins in the past 6 months, with more frequent logins suggesting a positive user experience and strong interest in the platform.

In [398]:
# H0: there is no difference between T & C in average of logins 
# H1: there is a difference between T & C in avergae of logins

In [ ]:
df_client_joined_cleaned

In [ ]:
df_Test_log = df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Test"]["logons_6_mnth"]
df_Control_log= df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Control"]["logons_6_mnth"]
st.ttest_ind(df_Test_log,df_Control_log, equal_var=False)

In [404]:
test_log= df_client_joined_cleaned[df_client_joined_cleaned['Variation']=='Test']['logons_6_mnth']
control_log= df_client_joined_cleaned[df_client_joined_cleaned['Variation']=='Control']['logons_6_mnth']

M1_log = test_log.mean() 
M2_log = control_log.mean()  
SD1_log = test_log.std(ddof=1) 
SD2_log = control_log.std(ddof=1)  
n1_log = len(test_log) 
n2_log = len(control_log)  

In [ ]:
SD_pooled = np.sqrt(((n1_log - 1) * SD1_log**2 + (n2_log - 1) * SD2_log**2) / (n1_log + n2_log - 2))  #cohens d
cohens_d = (M1_log - M2_log) / SD_pooled
cohens_d

In [408]:
# p-values: < 0.05 : there is a difference
# effectsize: -0.02: very small, not relevant

## 3. KPI: User Satisfaction Based on Balance Amount
Description: Gauge user satisfaction by examining the balance amount of a user's account, as a higher balance can indicate a positive financial experience and trust in the platform.

In [411]:
# KPI balance 
#H0: no difference in balance
#H1: difference in balance     -> p-value > 0.05: don't reject HO! 

In [ ]:
df_Test_bal = df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Test"]["balance"]
df_Control_bal= df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Control"]["balance"]
st.ttest_ind(df_Test_bal,df_Control_bal, equal_var=True)

## 4. KPI: User Satisfaction Based on the Number of Accounts
Description: Assess user satisfaction by counting the number of accounts a user holds on the platform, as a higher account number may reflect stronger engagement and satisfaction with the services offered.

In [416]:
# H0: no difference in account number
# H1: difference in account number      

In [ ]:
df_Test_acc = df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Test"]["num_accts"]
df_Control_acc= df_client_joined_cleaned[df_client_joined_cleaned["Variation"]=="Control"]["num_accts"]
st.ttest_ind(df_Test_acc,df_Control_acc, equal_var=False)

In [420]:
# effectsize:
control_acc= df_client_joined_cleaned[df_client_joined_cleaned['Variation']=='Control']["num_accts"]
control_acc
test_acc= df_client_joined_cleaned[df_client_joined_cleaned['Variation']=='Test']["num_accts"]
test_acc

M1_acc = test_acc.mean() 
M2_acc = control_acc.mean()  
SD1_acc = test_acc.std(ddof=1) 
SD2_acc = control_acc.std(ddof=1)  
n1_acc = len(test_acc) 
n2_acc = len(control_acc)  

In [ ]:
SD_pooled = np.sqrt(((n1_acc - 1) * SD1_acc**2 + (n2_acc - 1) * SD2_acc**2) / (n1_acc + n2_acc - 2))
cohens_d = (M1_acc - M2_acc) / SD_pooled
cohens_d

In [424]:
# p-value (slightly) < 0.05: reject H0
# effectsize: -0.018 : not relevant 

In [426]:
 # change over time 

In [ ]:
df_concat_joined    #joined df with month?

In [ ]:
df_concat_joined['year_month'].value_counts()

In [ ]:
monthly_visits_variation = df_concat_joined.groupby(['Variation', 'year_month']).size().reset_index(name='visit_count')
monthly_visits_variation

In [ ]:
control_data = monthly_visits_variation[monthly_visits_variation['Variation'] == 'Control'].set_index('year_month')
test_data = monthly_visits_variation[monthly_visits_variation['Variation'] == 'Test'].set_index('year_month')
percentage_diff = ((test_data['visit_count'] - control_data['visit_count']) / control_data['visit_count']) * 100
percentage_diff #difference in visits between groups

In [ ]:
df_concat_joined

In [ ]:
# error rate
df_concat_joined = df_concat_joined.sort_values(by=['client_id', 'date_time'])
df_concat_joined['previous_step'] = df_concat_joined.groupby('client_id')['process_step'].shift(1) #shifts values on cell down 
df_concat_joined

In [457]:
step_mapping = {'start': 0, 'step1': 1, 'step2': 2, 'step3': 3, 'confirm': 4}
df_concat_joined['process_step_numeric'] = df_concat_joined['process_step'].map(step_mapping)
df_concat_joined = df_concat_joined.sort_values(by=['client_id', 'date_time'])
df_concat_joined['previous_step_numeric'] = df_concat_joined.groupby('client_id')['process_step_numeric'].shift(1)
df_concat_joined['is_error'] = df_concat_joined['process_step_numeric'] < df_concat_joined['previous_step_numeric']

In [ ]:
print(df_concat_joined[['client_id', 'date_time', 'process_step', 'previous_step_numeric', 'is_error']])
